In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

data_list = []

# Arabam.com üzerindeki BMW ikinci el otomobil ilanlarına yönelik 19 sayfalık bir döngü
for page_number in range(1, 20):
    url = f"https://www.arabam.com/ikinci-el/otomobil/bmw?take=50&page={page_number}"
    r = requests.get(url)
    soup = BeautifulSoup(r.content, "lxml")
    listings = soup.find_all("tr", attrs={"class": "listing-list-item pr should-hover bg-white"})
    
    # Her bir ilanın detaylarına yönelik bir döngü
    for listing in listings:
        listing_links = listing.find("td", attrs={"class": "listing-modelname pr"})
        link_end = listing_links.a.get("href")
        link_start = "https://www.arabam.com/"
        link = link_start + link_end

        details = requests.get(link)
        details_soup = BeautifulSoup(details.content, "lxml")

        car_data = {}

        # Araba teknik bilgileri çekme
        car_info = details_soup.find_all("div", attrs={"class": "product-properties"})
        for technical_info in car_info:
            details_list = technical_info.find_all("div", attrs={"class": "property-item"})
            for detail in details_list:
                try:
                    description = detail.find("div", attrs={"class": "property-A"}).text
                    information = detail.find("div", attrs={"class": "property-value"}).text
                    information = information.strip()
                    car_data[description] = information
                except:
                    continue  
                    
        # Araba fiyat bilgilerini çekme
        car_prices = details_soup.find_all("div", attrs={"class": "product-properties"})
        for price_info in car_prices:
            prices_list = price_info.find_all("div", attrs={"class": "product-price-container"})
            for price in prices_list:
                description = price.find("div", attrs={"class": "property-key"}).text
                information = price.find("div", attrs={"class": "product-price"}).text
                information = information.strip()
                car_data[description] = information

        # Oluşturulan veri sözlüğünü listeye ekleme
        data_list.append(car_data)

# Veri listesini pandas DataFrame'e dönüştürme
df = pd.DataFrame(data_list)

# Sütun isimlerini temizleme
df.columns = df.columns.map(str.strip)

# Elde edilen verileri Excel dosyasına kaydetme
df.to_excel("car_listings_data.xlsx", index=False)
